In [1]:
import pandas as pd
import numpy as np
import datetime

In [3]:
class analyse_marjane:

    def __init__(self, customer_id, extraction_date):
        self.welcome_msg = "init marjane analyse"
        self.customer_id = customer_id
        self.extraction_date = extraction_date
    
    def load_trx_log(self, file_name):
        df = pd.read_parquet(f"{file_name}")
        ## convert data types
        df['item_qty'] = pd.to_numeric(df['item_qty'])
        df['item_price'] = pd.to_numeric(df['item_price'])
        df['trx_total_price'] = pd.to_numeric(df['trx_total_price'])
        df['trx_discount'] = pd.to_numeric(df['trx_discount'])
        df['trx_date'] = pd.to_datetime(df['trx_date'])
        file_nb_records = len(df)
        print(f"trx log file {file_name} \nloaded with : # {file_nb_records} records ...")
        ## update class status
        self.trx_log = df
        self.is_data_loaded = True
        return df
    
    def set_trx_log(self, trx_log):
        self.trx_log = trx_log
        self.is_data_loaded = True

    def getSpendByCategory(self, topk):
        spend_by_category = self.trx_log.groupby('item_category').agg({'item_price':[sum]}).reset_index()
        spend_by_category.columns = ['item_category', 'category_spend']
        spend_by_category.sort_values(by='category_spend', ascending=False, inplace=True)
        spend_by_category_reduced = spend_by_category[:topk]
        return spend_by_category_reduced
        
    def getDayOfWeekSplit(self):
    ## append DoW : 0 monday, 6 sunday
        tmp_df = self.trx_log.copy()
        tmp_df['trx_dow'] = tmp_df['trx_date'].dt.dayofweek
        tmp_df.drop_duplicates(subset=['trx_id'])['trx_dow'].value_counts(normalize=False, ascending=False)
        dow_pct = tmp_df.drop_duplicates(subset=['trx_id'])['trx_dow'].value_counts(normalize=True, ascending=False).to_frame().reset_index()
        dow_pct.columns = ['trx_dow', 'cnt_trx']
        return dow_pct


    def getTripStats(self):
        pass
        
        

In [33]:
## instantiate objects

analyse_mm = analyse_marjane(customer_id="130577", extraction_date="2022_07_22")
df_mm = analyse_mm.load_trx_log(file_name = "df_orders_MM_07_24_2022.parquet")

trx log file df_orders_MM_07_24_2022.parquet 
loaded with : # 536 records ...


In [6]:
print(analyse_mm.customer_id)

130577


In [7]:
spend_categ = analyse_mm.getSpendByCategory(20)

In [8]:
spend_categ.head(10)

,item_category,category_spend
47,PETIT ELECTRO,1147.00
23,ENTRETIEN,757.90
31,HYGIENE,695.25
12,BOISSONS CHAUDES,595.90
4,ART MENAG/RANG,493.60
16,CAPILLAIRE,433.00
19,CONSERVES & PLATS CUISINES,411.60
40,LINGE DE MAISON,378.90
21,DROGUERIE,336.45
50,POULET L.S,328.00


In [34]:
dow_distrib = analyse_mm.getDayOfWeekSplit()

In [35]:
dow_distrib

,trx_dow,cnt_trx
0,6,0.228571
1,2,0.171429
2,5,0.142857
3,4,0.142857
4,1,0.114286
5,3,0.114286
6,0,0.085714
